## Academic exercise for study

### Environment installation and configuration

In [1]:
# Python interface to Spark
!pip install pyspark --quiet
# Installation and update of the PyDrive library, for interacting with Google Drive using Python.
!pip install -U -q PyDrive --quiet
# Install OpenJDK 8
!apt install openjdk-8-jdk-headless &> /dev/null
# Download the ngrok zip file to access the local server over the internet
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
# Unzip the ngrok zip file
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
# Starts ngrok, allowing HTTP traffic on port 4050
get_ipython().system_raw('./ngrok http 4050 &')
# Import the Python os module
import os
# Sets the JAVA_HOME environment variable to the location of Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### Setting up and starting a Spark session using the PySpark library

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[2]")
sc = SparkSession.builder.config(conf=conf).getOrCreate()

### Load data

In [3]:
# Download from http to local file
!wget --quiet --show-progress http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-03-09/visualisations/listings.csv

listings.csv        100%[===================>]   1.16M  3.15MB/s    in 0.4s    


In [4]:
# Upload data from Airbnb
df_spark = sc.read.csv("/content/listings.csv", inferSchema=True, header=True)
# See some information about the data types of each column
df_spark.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- number_of_reviews_ltm: string (nullable = true)
 |-- license: string (nullable = true)



### Spark SQL operations

In [5]:
# Register the DataFrame as a temporary table
df_spark.createOrReplaceTempView("airbnb")
# Perform a SQL query
sql_query = """
    SELECT name, price
    FROM airbnb
    WHERE price <= 100
"""
# Execute the SQL query
results = sc.sql(sql_query)
# Show the results
results.show()

+--------------------+-----+
|                name|price|
+--------------------+-----+
|Quiet Garden View...|   69|
|Comfortable doubl...|   76|
|Comfortable singl...|   56|
|Bed & Breakfast i...|   95|
|Private Room at o...|   99|
|Beautiful and spa...|   55|
|nice cozy room ne...|   60|
|Clean & Cozy room...|   86|
|Central apartment...|  100|
|Amsterdam sweetho...|   56|
|Nice and Cozy Roo...|   55|
|cosy room with pr...|   80|
|Charming Studio w...|   88|
|Unique Beautiful ...|   88|
|SunFlower, A Sunn...|   97|
|Spacious studio a...|  100|
|A'dam West - room...|   69|
|B&B WestergasTver...|   80|
|Southern Comfort ...|   90|
|Lovely bright roo...|   75|
+--------------------+-----+
only showing top 20 rows



In [6]:
# Count the number of listings in each neighborhood:
sql_query = """
    SELECT neighbourhood, COUNT(*) AS count
    FROM airbnb
    GROUP BY neighbourhood
    ORDER BY count DESC
"""
results = sc.sql(sql_query)
# rdd
rdd_count = df_spark.rdd
neighborhood_count = rdd_count.map(lambda row: (row.neighbourhood, 1)) \
                        .reduceByKey(lambda x, y: x + y) \
                        .sortBy(lambda kv: kv[1], ascending=False)
# Output
neighborhood_count.collect()

[('De Baarsjes - Oud-West', 1136),
 ('Centrum-West', 921),
 ('De Pijp - Rivierenbuurt', 802),
 ('Centrum-Oost', 680),
 ('Westerpark', 481),
 ('Zuid', 464),
 ('Oud-Oost', 408),
 ('Bos en Lommer', 344),
 ('Oud-Noord', 321),
 ('Oostelijk Havengebied - Indische Buurt', 260),
 ('Watergraafsmeer', 207),
 ('Noord-West', 179),
 ('IJburg - Zeeburgereiland', 157),
 ('Slotervaart', 143),
 ('Noord-Oost', 111),
 ('Geuzenveld - Slotermeer', 87),
 ('Buitenveldert - Zuidas', 80),
 ('De Aker - Nieuw Sloten', 48),
 ('Bijlmer-Centrum', 44),
 ('Gaasperdam - Driemond', 44),
 ('Osdorp', 43),
 ('Bijlmer-Oost', 28),
 (None, 11),
 ('52.40679', 1),
 ('52.361416', 1),
 ('52.33857', 1),
 ('52.36189', 1),
 ('52.36628', 1),
 ('52.344265', 1),
 ('52.3890549', 1),
 ('52.39808', 1),
 ('52.3657547', 1)]

In [7]:
# Find the host with the most listings:
sql_query = """
    SELECT host_id, host_name, COUNT(*) AS listing_count
    FROM airbnb
    GROUP BY host_id, host_name
    ORDER BY listing_count DESC
    LIMIT 1
"""
results = sc.sql(sql_query)
results.show()

+---------+---------+-------------+
|  host_id|host_name|listing_count|
+---------+---------+-------------+
|203731852|   Sweets|           21|
+---------+---------+-------------+



In [10]:
# ID with the most listings

# Convert to rdd
rdd2 = df_spark.rdd
# each row to a function that returns a tuple
# count number of lists
# Descending order based on the listing count value
# first element of RDD
host_listing_count = rdd2.map(lambda row: (row.host_id, 1))\
                        .reduceByKey(lambda x, y: x + y)\
                        .sortBy(lambda kv: kv[1], ascending=False)\
                        .first()

host_listing_count

('203731852', 21)